In [3]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly

! pip install tensorflow-gpu==2.0.0-beta1

! pip install tfkerassurgeon


Uninstalling tensorflow-gpu-2.0.0b1:
  Successfully uninstalled tensorflow-gpu-2.0.0b1
Uninstalling tfkerassurgeon-0.2.1:
  Successfully uninstalled tfkerassurgeon-0.2.1
  Using cached https://files.pythonhosted.org/packages/2b/53/e18c5e7a2263d3581a979645a185804782e59b8e13f42b9c3c3cfb5bb503/tensorflow_gpu-2.0.0b1-cp36-cp36m-manylinux1_x86_64.whl


In [4]:
import numpy as np

import tensorflow as tf

from tensorflow.python.keras import layers
from tensorflow.python.keras import activations
from tensorflow.python.keras import models
from tensorflow.python.keras import backend
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.utils.np_utils import to_categorical

import tfkerassurgeon
from tfkerassurgeon import identify
from tfkerassurgeon.operations import delete_channels

print(tf.__version__)


2.0.0-beta1


In [0]:

# Set some static values that can be tweaked to experiment
keras_verbosity = 2 # limits the printed output but still gets the Epoch stats
epochs=200 # we'd never reach 200 because we have early stopping
batch_size=128 # tweak this depending on your hardware and Model


In [0]:

# Load the dataset (it will automatically download it if needed), they provided a nice helper that does all the network and downloading for you
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# This is an leterantive to the MNIST numbers dataset that is a computationlally harder problem
#(X_train, Y_train), (X_test, Y_test) = keras.datasets.fashion_mnist.load_data()

# we need to make sure that the images are normalized and in the right format
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# expand the dimensions to get the shape to (samples, height, width, channels) where greyscale has 1 channel
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# one-hot encoding, this way, each digit has a probability output
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

In [0]:

# Simple reusable shorthand to compile the model, so that we can be sure to use the same optomizer, loss, and metrics
def compile_model(model):
    
    model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])



In [0]:

# method that encapsulates the Models archeteture and construction
def build_model():

    # Create LeNet model
    model = models.Sequential()
    model.add(layers.Conv2D(20,
                     [3, 3],
                     input_shape=[28, 28, 1],
                     activation='relu',
                     name='conv_1'))
    model.add(layers.MaxPool2D())
    model.add(layers.Conv2D(50, [3, 3], activation='relu', name='conv_2'))
    model.add(layers.MaxPool2D())
    model.add(layers.Permute((2, 1, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu', name='dense_1'))
    model.add(layers.Dense(10, activation='softmax', name='dense_2'))

    compile_model(model)

    return model


In [9]:

# a simple method that gets the callbacks for training
def get_callbacks(use_early_stopping = True, use_reduce_lr = True):

    callback_list = []

    if(use_early_stopping):

        callback_list.append(callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=0,
                                             patience=10,
                                             verbose=keras_verbosity,
                                             mode='auto'))

    if(use_reduce_lr):

        callback_list.append(callbacks.ReduceLROnPlateau(monitor='val_loss',
                                            factor=0.1,
                                            patience=5,
                                            verbose=keras_verbosity,
                                            mode='auto',
                                            epsilon=0.0001,
                                            cooldown=0,
                                            min_lr=0))

    return callback_list

# and get the callbacks
callback_list = get_callbacks()

W0802 02:05:03.868893 140189758031744 callbacks.py:1781] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


In [0]:

# Simple reusable shorthand for evaluating the model on the Validation set 
def fit_model(model):
    
    return model.fit(
                    X_train,
                    Y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=keras_verbosity,
                    validation_data=(X_test, Y_test),
                    callbacks=callback_list)


In [0]:

# Simple reusable shorthand for evaluating the model on the Validation set 
def eval_model(model):

    return model.evaluate(
                        X_test, 
                        Y_test, 
                        batch_size=batch_size, 
                        verbose=keras_verbosity)


In [0]:







# THIS IS WHERE THE MAGIC HAPPENS!
# This method uses the Keras Surgeon to identify which parts od a layer can be pruned and then deletes them
# Note: it returns the new, pruned model, that was recompiled
def prune_layer(model, layer):
    
    # Get the APOZ (Average Percentage of Zeros) that should identify where we can prune
    apoz = identify.get_apoz(model, layer, X_test)

    # Get the Channel Ids that have a high APOZ, which indicates they can be pruned
    high_apoz_channels = identify.high_apoz(apoz)

    # Run the pruning on the Model and get the Pruned (uncompiled) model as a result
    model = delete_channels(model, layer, high_apoz_channels)

    # Recompile the model
    compile_model(model)

    return model

  
  
  
  
  
  
  


In [0]:

# A helper that gets the layer by it's name 
def prune_layer_by_name(model, layer_name):

    # First we get the layer we are working on
    layer = model.get_layer(name=layer_name)
    # Then prune is and return the pruned model
    return prune_layer(model, layer)


In [0]:

# the main function, that runs the training
def main(): 

    
    # build the model
    model = build_model()

    # Initial Train on dataset
    results = fit_model(model)

    # eval and print the results of the training
    loss = eval_model(model)
    print('original model loss:', loss, '\n')
    
    # NOTE: This while true will continue until it ERRORs out because there is no escape condition.
    while True:

        # only prune the Dense layer for this example
        layer_name = 'dense_1'
        # Run the Pruning on the layer
        model = prune_layer_by_name(model, layer_name)

        # eval and print the results of the pruning
        loss = eval_model(model)
        print('model loss after pruning: ', loss, '\n')
        
        # Retrain the model to accomodate for the changes
        results = fit_model(model)

        # eval and print the results of the retraining
        loss = eval_model(model)
        print('model loss after retraining: ', loss, '\n')

        # While TRUE will repeat until an ERROR occurs


In [15]:

# Run the main Method
if __name__ == '__main__':
    main()


W0802 02:05:04.909610 140189758031744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 - 4s - loss: 0.1878 - accuracy: 0.9465 - val_loss: 0.0492 - val_accuracy: 0.9840
Epoch 2/200
60000/60000 - 2s - loss: 0.0499 - accuracy: 0.9847 - val_loss: 0.0335 - val_accuracy: 0.9890
Epoch 3/200
60000/60000 - 2s - loss: 0.0351 - accuracy: 0.9888 - val_loss: 0.0352 - val_accuracy: 0.9888
Epoch 4/200
60000/60000 - 2s - loss: 0.0259 - accuracy: 0.9917 - val_loss: 0.0302 - val_accuracy: 0.9906
Epoch 5/200
60000/60000 - 2s - loss: 0.0185 - accuracy: 0.9937 - val_loss: 0.0298 - val_accuracy: 0.9893
Epoch 6/200
60000/60000 - 2s - loss: 0.0146 - accuracy: 0.9954 - val_loss: 0.0326 - val_accuracy: 0.9901
Epoch 7/200
60000/60000 - 2s - loss: 0.0126 - accuracy: 0.9961 - val_loss: 0.0388 - val_accuracy: 0.9890
Epoch 8/200
60000/60000 - 2s - loss: 0.0097 - accuracy: 0.9966 - val_loss: 0.0377 - val_accuracy: 0.9892
Epoch 9/200
60000/60000 - 2s - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.0328 - val_accuracy: 0.9903
Epoch

TypeError: ignored